In [1]:
# -*- coding: utf-8 -*-
"""
Created on 2024/8/15 
@author: jasonting
Optimized 840_OCT_3_v2: read multiple folders
"""
import os,glob
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from scipy.fftpack import fft, ifft
from scipy.interpolate import CubicSpline
import time
'''
1. 一次讀一個frame.dat，所以記憶體佔用會小很多
2. 處理完後輸出成一張.png影像與一個.npy檔案，分別存在result_image與result_npy裡
兩種儲存方式的差別在於integer或float，integer這樣資料會小很多

'''
#路徑可動態調整

path_background = r'E:\OCT\dental OCT\bare tooth\ensemble_model_aug\code\2024_8_13\BG'
 

# path_file = r'D:\OCT\dental OCT\bare tooth\ensemble_model_aug\code\2024_8_13\cal_raw'

path_file = r'E:\OCT\dental OCT\bare tooth\ensemble_model_aug\code\2024_8_13\nor_raw'



# 獲取目錄中的子資料夾列表
subfolders = sorted( [f for f in os.listdir(path_file) if os.path.isdir(os.path.join(path_file, f))] )

# 按子資料夾名稱中的數字部分進行排序: 因為python的listdir排序不完全依照數字大小排 ex:14 15 2 25 3 37...
subfolders_sorted = sorted(subfolders, key=lambda x: int(x[1:]))

# 选择指定子文件夹 : 改index決定要讀取哪些subfolder
selected_subfolders = subfolders_sorted[:10] #讀取首個至10th folder





#畫圖處可有可無，故註解
# def plot_waveform(mean_BG):
#     plt.plot(mean_BG, linewidth=0.5)   # 一維所以直接 plot 他會自動抓x跟y
#     plt.show()

lines_per_frame = 2000 #共幾條A-line
total_length = 2048  # one A-line = 2048 pixels

number_BG = 10 #注意看有幾張BG，不夠的話要加到10

#根據folder的dat檔案數量調整:可以調超過frame的總數，不過跑完所有frame後找不到下一個會出現FileNotFoundError
# number_OCT = 1000 #frame數量，由於每個folder的資料數目不一: 想辦法以一個表達式表示所有folder中的資料數目
max_number_OCT = 1000

# ============== BG subtraction ==============#True False
BG_subtraction = True

if BG_subtraction is True:
    data_10BG = np.zeros((lines_per_frame, total_length, number_BG)) #初始化三維矩陣，儲存10張2000 * 2048 的BG
    for num in range(1, number_BG +1):  #range(1, 11) 從 1 開始到 10
        file_background = os.path.join(path_background, 'frame%d.dat' % num) #
        temp = np.fromfile(file_background, dtype='>i2')  #讀取背景影像的二進位數據: i2> 指定為16位元整數
        data_10BG[:, :, num-1] = temp[4:].reshape(2000, 2048)   # 把多的前4筆資料移除後，剩下的數據重塑為 2000x2048矩陣，並存入 data_10BG 
    mean_BG = np.mean(data_10BG, axis=2) #計算多張背景影像的平均值(降噪)
    A_line_mean_BG = np.mean(np.mean(data_10BG.reshape((2000, 2048, 10)), axis=2), axis=0).reshape(2048, 1) #計算每一行的平均值，得到一個長度為 2048 的一維向量，表示平均背景的 A-line
    # plot_waveform(20*np.log10(np.abs(np.fft.fft(A_line_mean_BG[:, 0]))))

    matrix_mean_BG = np.tile(A_line_mean_BG, 2000).T    # 將 A_line_mean_BG 擴增為與影像大小一致的二維矩陣，方便後續處理
    # plot_waveform(A_line_mean_BG) 

elif BG_subtraction is False:
    matrix_mean_BG = 0
# ============== BG subtraction ==============#


# ==============load pixel calibration .txt，必須與現正執行的840_OCT_3_v2.ipynb檔放在同路徑  ==============#


f=open(r'E:\OCT\dental OCT\bare tooth\ensemble_model_aug\code\2024_8_13\dat_data_preprocessing\p_avg_1.txt','r')
p_avg_txt=f.readlines()
p_avg = np.zeros(len(p_avg_txt))  # 建立空矩陣儲存資料  
p_avg[:] = p_avg_txt[:]   # 把前面讀進來的data塞進 numpy矩陣裡 
base_line = np.arange(4096)


#確認儲存結果的路徑是否存在，若不存在則建立
final_OCT = np.zeros((2000, 550, max_number_OCT))
test_fft = np.zeros((2000, 2048), dtype='complex_')
##############################################################################################################


#前處理

# 最外層添加for迴圈，依序跑自己想前處理的folders(用selected_subfolders搭配slice決定範圍)
for subfolder in selected_subfolders:
    subfolder_path = os.path.join(path_file, subfolder)
    npy_save = os.path.join(subfolder_path, 'npy')
    png_save = os.path.join(subfolder_path, 'png')

    print(f"Processing folder: {subfolder_path}")

    # 创建保存目录
    Path(npy_save).mkdir(parents=True, exist_ok=True)
    Path(png_save).mkdir(parents=True, exist_ok=True)

    # 客製化当前子文件夹中的 .dat 文件数量:可以針對存取不同資料數目的folder進行檔案讀取而不發生error
    number_OCT = len(glob.glob(os.path.join(subfolder_path, 'frame*.dat')))
    print(f"Number of frames in {subfolder}: {number_OCT}")
    ##############################################################################################################

    ''' 開始一個frame一個frame讀入，並將處理完的結果儲存為圖片(整數矩陣)與npy(float矩陣)
    '''
    start = time.process_time()
    for num in range(0, number_OCT):  # 從 0 開始到 frame的數量-1
        ''' 將會用到的矩陣變數先建立，同時用於每次完成迴圈後清空
        '''
        if num%50 == 0:
            print(f'Folder {subfolder}: Processed {num} out of {number_OCT}.')  #監督執行進度
        data_OCT = np.zeros((lines_per_frame, total_length))    # 一個frame輸出一次結果，因此只需要建立二維矩陣
        temp = np.zeros(2000*2048)
        OCT_interplation = np.zeros((2000, 4096), dtype='complex_')
        zero_padding = np.zeros((2000, 4096), dtype='complex_')
        inverse_ZeroPadding = np.zeros((2000, 2048), dtype='complex_') #why?
        fig_reshape = np.zeros((550, 2000))
        file_background = os.path.join(subfolder_path, 'frame%d.dat' % (num +1)) #修
        temp = np.fromfile(file_background, dtype='>i2')  #讀取背景影像的二進位數據: i2> 指定為16位元整數
        data_OCT = temp[4:].reshape(2000, 2048) - matrix_mean_BG   # 把多的前4筆資料移除(標頭檔)並減去mean_BG後存入data_OCT
        # plot_waveform(data_OCT[0, :, num])
        
        temp_fft = fft(data_OCT)   #對 OCT 影像的每個 A-line 進行 FFT，將影像轉換到頻域，以便處理 
        
        # 將zero padding加入fft中間，提高頻率解析度
        zero_padding[:, :1024] = temp_fft[:, :1024]
        zero_padding[:, 3072:] = temp_fft[:, 1024:]
        
        temp_ifft = ifft(zero_padding) #轉回空間域
        # plot_waveform(temp_ifft[0, :])
        
        # 因為內插只能一維一維做，因此多一層for迴圈分別完成每一列的內差
        for num_interplation in range(lines_per_frame): 
            cubic_interplation = CubicSpline(p_avg, temp_ifft[num_interplation, :]) #將 temp_ifft 的頻域數據從原始點 p_avg 內插到新的點: cubic_interplation(base_line)
            OCT_interplation[num_interplation, :] = cubic_interplation(base_line).flatten() #將內插後的結果存入 OCT_interplation 中，這樣每一行的數據都在一個均勻的基準上進行內插
            # if num_interplation == 0:
            #     plot_waveform(OCT_interplation[0, :])

    
        temp_interplation_fft = 20*np.log10 (np.abs(fft(OCT_interplation))) #對內插後的影像進行FFT，並計算影像在頻域的強度(以dB為單位)
        #plot_waveform(temp_interplation_fft[0, :])
        
        # 重新分配頻域圖形，左右對調(方便做ifft)
        inverse_ZeroPadding[:, :1024] = temp_interplation_fft[:, 3072:]
        inverse_ZeroPadding[:, 1024:] = temp_interplation_fft[:, :1024]
        
        #plot_waveform(inverse_ZeroPadding[0, :])
        
        final_OCT = np.abs(inverse_ZeroPadding[:, 450:1000])    # 提取所關注區域，取絕對值以獲取最終的浮點數型別數據 

        fig_reshape = np.rot90(final_OCT) #轉90度，適應顯示分析要求
        

        #保存结果 
        plt.imsave(png_save + '/frame%d.png' %(num+1), fig_reshape, cmap = 'gray', vmin=50, vmax=80)
        np.save(npy_save + '/frame%d.npy' %(num+1),fig_reshape)

    end = time.process_time()
    print(f"Folder {subfolder} execution time: {end - start} seconds")
print("All folders processed.")

Processing folder: D:\OCT\dental OCT\bare tooth\ensemble_model_aug\code\2024_8_13\cal_raw\c21
Processing folder: D:\OCT\dental OCT\bare tooth\ensemble_model_aug\code\2024_8_13\cal_raw\c22
Processing folder: D:\OCT\dental OCT\bare tooth\ensemble_model_aug\code\2024_8_13\cal_raw\c23
Processing folder: D:\OCT\dental OCT\bare tooth\ensemble_model_aug\code\2024_8_13\cal_raw\c26
Processing folder: D:\OCT\dental OCT\bare tooth\ensemble_model_aug\code\2024_8_13\cal_raw\c30
Folder c30: Processed 0 out of 1000.
Folder c30: Processed 50 out of 1000.
